# Statistical Test for alpha variant paper

In [1]:
#basic data libraries
import makedalytics as ma
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from matplotlib import rc
from datetime import datetime
from datetime import date
from pandas.tseries.offsets import DateOffset
# math libraries and packages
import math as mth
import scipy
import scipy.special
import scipy.stats as stats
#fancy visualization and other fancy libraries and packages
import datetime as dt
from IPython.display import Markdown, display
from matplotlib import pylab
import pylab as p

In [2]:
#import MOH dataset
moh= pd.read_csv('base_MOH_data/covid19-data-israel-30032021.csv')

In [3]:
moh =  moh.rename(columns={'מספר חולים קשה': "number_kashim", 'מספר חולים מונשמים': "total_intubated", 'מספר נפטרים מצטבר':'deaths'})

In [4]:
moh =  moh.rename(columns={'תאריך': "date"})
moh =  moh.rename(columns={'מספר בדיקות מצטבר': "number_tests", 'חולים מאושפזים': "number_admitted",'מספר חולים מצטבר':"number_sick"})

moh['total_deaths_n'] = moh['deaths'].str.replace(',', '')
moh['total_deaths_n'] = pd.to_numeric(moh['total_deaths_n'])
# now we can add a column for daily deaths by subtracting the total deaths from the deaths the day before
moh['daily_deaths']= moh['total_deaths_n']-  moh['total_deaths_n'].shift()

In [6]:
moh

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths
0,26/01/2020 00:00:00,134,0,0,7,0,0,0,NaN
1,27/01/2020 00:00:00,135,0,0,7,0,0,0,0.0
2,28/01/2020 00:00:00,135,0,0,7,0,0,0,0.0
3,29/01/2020 00:00:00,141,0,0,7,0,0,0,0.0
4,30/01/2020 00:00:00,145,0,0,7,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...
425,26/03/2021 00:00:00,"14,828,138",718,207,"831,625",458,"6,158",6158,11.0
426,27/03/2021 00:00:00,"14,844,984",668,205,"831,856",442,"6,164",6164,6.0
427,28/03/2021 00:00:00,"14,856,471",680,210,"831,985",452,"6,176",6176,12.0
428,29/03/2021 00:00:00,"14,898,839",713,202,"832,348",441,"6,185",6185,9.0


In [7]:
moh ['date_as_date'] = pd.to_datetime(moh.date, format= '%d/%m/%Y %H:%M:%S')
moh_cut_for_calc = moh[moh.date_as_date < '02/02/2021']
moh_cut_for_calc.tail(60)

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date
313,04/12/2020 00:00:00,"6,291,210",704,92,"342,985",324,"2,920",2920,10.0,2020-12-04
314,05/12/2020 00:00:00,"6,317,085",705,89,"343,985",335,"2,930",2930,10.0,2020-12-05
315,06/12/2020 00:00:00,"6,360,013",735,83,"345,347",343,"2,938",2938,8.0,2020-12-06
316,07/12/2020 00:00:00,"6,427,680",729,84,"347,206",335,"2,945",2945,7.0,2020-12-07
317,08/12/2020 00:00:00,"6,498,175",736,89,"348,974",327,"2,957",2957,12.0,2020-12-08
318,09/12/2020 00:00:00,"6,573,645",745,90,"350,841",321,"2,970",2970,13.0,2020-12-09
319,10/12/2020 00:00:00,"6,651,462",764,95,"352,846",320,"2,983",2983,13.0,2020-12-10
320,11/12/2020 00:00:00,"6,728,672",772,99,"354,681",339,"2,995",2995,12.0,2020-12-11
321,12/12/2020 00:00:00,"6,773,532",779,101,"355,978",343,"3,011",3011,16.0,2020-12-12
322,13/12/2020 00:00:00,"6,825,007",865,99,"357,707",378,"3,020",3020,9.0,2020-12-13


In [9]:
moh_cut_for_calc ['date_as_date'] = pd.to_datetime(moh_cut_for_calc.date, format= '%d/%m/%Y %H:%M:%S')
moh_cut_for_calc['total_sick_n']= moh_cut_for_calc.number_sick.str.replace(',', '')
moh_cut_for_calc['total_sick_n'] = pd.to_numeric(moh_cut_for_calc['total_sick_n'])
#moh['case_fatality_rate'] = moh.total_deaths_n/moh.total_sick_n
moh_cut_for_calc['daily_sick'] = moh_cut_for_calc['total_sick_n'] - moh_cut_for_calc['total_sick_n'].shift()
#monthly_moh = lo.resample('M').sum()

In [10]:
moh_cut_for_calc[moh_cut_for_calc.date_as_date == '21/12/2020']

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date,total_sick_n,daily_sick
330,21/12/2020 00:00:00,"7,455,432","1,103",112,"380,236",485,"3,145",3145,16.0,2020-12-21,380236,3851.0


## Note: this spreadsheet has been touched by Excel. Dates are badly formatted sometimes in US, sometimes in EU style:
Therefore I print out the rows to show the dates and values so they can be check.
Curse Microsoft. 

In [18]:
moh_cut_for_calc[moh_cut_for_calc.date_as_date == pd.to_datetime('12/07/2020')]

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date,total_sick_n,daily_sick
316,07/12/2020 00:00:00,"6,427,680",729,84,"347,206",335,"2,945",2945,7.0,2020-12-07,347206,1859.0


In [11]:
moh_cut_for_calc[moh_cut_for_calc.date_as_date == '12/07/2020']

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date,total_sick_n,daily_sick
316,07/12/2020 00:00:00,"6,427,680",729,84,"347,206",335,"2,945",2945,7.0,2020-12-07,347206,1859.0


In [17]:
moh_cut_for_calc[moh_cut_for_calc.date_as_date == pd.to_datetime('21/12/2020')]

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date,total_sick_n,daily_sick
330,21/12/2020 00:00:00,"7,455,432","1,103",112,"380,236",485,"3,145",3145,16.0,2020-12-21,380236,3851.0


In [12]:
moh_cut_for_calc[moh_cut_for_calc.date_as_date == '21/12/2020']

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date,total_sick_n,daily_sick
330,21/12/2020 00:00:00,"7,455,432","1,103",112,"380,236",485,"3,145",3145,16.0,2020-12-21,380236,3851.0


In [13]:
moh_cut_for_calc[moh_cut_for_calc.date_as_date == '27/12/2020']

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date,total_sick_n,daily_sick
336,27/12/2020 00:00:00,"7,977,930","1,380",147,"402,898",607,"3,262",3262,21.0,2020-12-27,402898,3528.0


In [16]:
moh_cut_for_calc[moh_cut_for_calc.date_as_date == pd.to_datetime('27/12/2020')]

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date,total_sick_n,daily_sick
336,27/12/2020 00:00:00,"7,977,930","1,380",147,"402,898",607,"3,262",3262,21.0,2020-12-27,402898,3528.0


In [14]:
moh_cut_for_calc[moh_cut_for_calc.date_as_date == '01/10/2021']

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date,total_sick_n,daily_sick
350,10/01/2021 00:00:00,"9,405,642","2,137",240,"493,689","1,074","3,717",3717,41.0,2021-01-10,493689,6936.0


In [15]:
moh_cut_for_calc[moh_cut_for_calc.date_as_date == pd.to_datetime('01/10/2021')]

,date,number_tests,מספר חולים מאושפזים,total_intubated,number_sick,number_kashim,deaths,total_deaths_n,daily_deaths,date_as_date,total_sick_n,daily_sick
350,10/01/2021 00:00:00,"9,405,642","2,137",240,"493,689","1,074","3,717",3717,41.0,2021-01-10,493689,6936.0


In [19]:
deaths_by_jan10th = moh_cut_for_calc.loc[moh_cut_for_calc.date_as_date == pd.to_datetime('01/10/2021'), 'total_deaths_n']
deaths_by_dec27th = moh_cut_for_calc.loc[moh_cut_for_calc.date_as_date == pd.to_datetime('27/12/2020'), 'total_deaths_n']

In [20]:
deaths_by_jan10th

350    3717
Name: total_deaths_n, dtype: int64

In [21]:
deaths_by_dec27th 

336    3262
Name: total_deaths_n, dtype: int64

In [22]:
print("deaths between december 27th and january 10th:", 3717-3262)
deaths_dec27_to_jan10 = 3717-3262

deaths between december 27th and january 10th: 455


In [23]:
cases_by_jan10th = moh_cut_for_calc.loc[moh_cut_for_calc.date_as_date == pd.to_datetime('01/10/2021'), 'number_sick']
cases_by_dec27th = moh_cut_for_calc.loc[moh_cut_for_calc.date_as_date == pd.to_datetime('27/12/2020'), 'number_sick']

In [24]:
cases_by_jan10th

350    493,689
Name: number_sick, dtype: object

In [25]:
cases_by_dec27th

336    402,898
Name: number_sick, dtype: object

In [26]:
print("cases between december 27th and january 10th:", 493689-402898)
cases_december_27_to_jan10 = 493689-402898

cases between december 27th and january 10th: 90791


In [27]:
print("cases minus deaths between december 27th and january 10th:", cases_december_27_to_jan10-deaths_dec27_to_jan10)
cases_minus_deaths_post = cases_december_27_to_jan10-deaths_dec27_to_jan10

cases minus deaths between december 27th and january 10th: 90336


In [31]:
deaths_by_dec21st = moh_cut_for_calc.loc[moh_cut_for_calc.date_as_date == pd.to_datetime('21/12/2020'), 'total_deaths_n']
deaths_by_dec7th = moh_cut_for_calc.loc[moh_cut_for_calc.date_as_date == pd.to_datetime('12/07/2020'), 'total_deaths_n']

In [32]:
deaths_by_dec21st

330    3145
Name: total_deaths_n, dtype: int64

In [33]:
deaths_by_dec7th

316    2945
Name: total_deaths_n, dtype: int64

In [34]:
print("deaths between december 7th and january 21th:", 3145-2945)
deaths_december7_to_21 = 3145-2945

deaths between december 7th and january 21th: 200


In [36]:
cases_by_dec21st = moh_cut_for_calc.loc[moh_cut_for_calc.date_as_date == pd.to_datetime('21/12/2020'), 'number_sick']
cases_by_dec7th = moh_cut_for_calc.loc[moh_cut_for_calc.date_as_date == pd.to_datetime('12/07/2020'),  'number_sick']

In [37]:
cases_by_dec21st

330    380,236
Name: number_sick, dtype: object

In [38]:
cases_by_dec7th

316    347,206
Name: number_sick, dtype: object

In [39]:
print("cases minus deaths between december 7th and dec 21st:", 380236-347206)
cases_december7_to_21 = 380236-347206

cases minus deaths between december 7th and dec 21st: 33030


In [40]:
deaths_december7_to_21/cases_december7_to_21

0.006055101422948834

In [41]:
deaths_dec27_to_jan10/cases_december_27_to_jan10

0.005011509951426903

In [42]:
living_cases_december27_to_j10 = cases_december_27_to_jan10 - deaths_dec27_to_jan10

living_cases_december7_to_d21 = cases_december7_to_21 - deaths_december7_to_21

# From these numbers we can do a Fisher's test with either deaths versus cases in these two time period, or deaths versus cases minus deaths.

In [43]:
ar=np.array([[deaths_december7_to_21, deaths_dec27_to_jan10],[living_cases_december7_to_d21, living_cases_december27_to_j10]])    
df=pd.DataFrame(ar, columns=["Before b117", "after b117"])
df.index=["deaths", "cases minus deaths"] 
df

,Before b117,after b117
deaths,200,455
cases minus deaths,32830,90336


In [44]:


df2=df.copy()
df2.loc['Column_Total']= df2.sum(numeric_only=True, axis=0)
df2.loc[:,'Row_Total'] = df2.sum(numeric_only=True, axis=1)
df2



,Before b117,after b117,Row_Total
deaths,200,455,655
cases minus deaths,32830,90336,123166
Column_Total,33030,90791,123821


In [45]:
# based on online Fisher tool:
print("The Fisher exact test statistic value is The Fisher exact test statistic value is 0.0268. The result is significant at p < .05.. The result is significant at p < .05.")
print("Courtesy of online tool at https://www.socscistatistics.com/tests/fisher/default2.aspx")
print("But scientists don't trust the internet...I'll do my own calculations. I'll start with an expected values table:")

The Fisher exact test statistic value is The Fisher exact test statistic value is 0.0268. The result is significant at p < .05.. The result is significant at p < .05.
Courtesy of online tool at https://www.socscistatistics.com/tests/fisher/default2.aspx
But scientists don't trust the internet...I'll do my own calculations. I'll start with an expected values table:


In [46]:
# expected values table
n=df2.at["Column_Total", "Row_Total"]  # grand total 

exp=df2.copy()               # create dataframe with expected frequencies
for x in exp.index[0:-1]:
    for y in exp.columns[0:-1]:
        # round expected values to nearest hundredths:
        v= (((df2.at[x, "Row_Total"]) * (df2.at["Column_Total", y])   )   /n ).round(2) 
        exp.at[x,y]=float(v)
print('expected contingency')
exp

expected contingency


,Before b117,after b117,Row_Total
deaths,174,480,655
cases minus deaths,32855,90310,123166
Column_Total,33030,90791,123821


In [49]:
oddsratio, pvalue = stats.fisher_exact([[deaths_december7_to_21, deaths_dec27_to_jan10],[living_cases_december7_to_d21, living_cases_december27_to_j10]])    
pvalue

0.026758222168445235

# or approximately ... close enough to zero

The probability that we would observe this or an even more imbalanced ratio by chance is about 2.68% percent by my calculations on the internet or in Python. The Fisher exact test statistic value is 0.0268. The result is significant at p < .05. A commonly used significance level is 5%–if we adopt that, we are defintely fine. 